# Import Lib

In [1]:
!pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11685 sha256=769d50c946959a65f9d162443298edef35c27a2fcac84c0aeaed22f345059b81
  Stored in directory: /root/.cache/pip/wheels/e4/26/c3/4f401d3e5c907d26a2c4ca8484141f02f7630c98fbcfd78a5e
Successfully built wikipedia


In [2]:
import json
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import time
import spacy
nlp = spacy.load('en_core_web_sm')

import pickle
from sklearn.metrics.pairwise import cosine_similarity
import math

import sys,os,os.path
import wikipedia

**Loading News Data**

In [3]:
raw_data = pd.read_json("/kaggle/input/signalmedia/sample-1M.jsonl",
                        lines=True,
                        orient='columns')
print(raw_data.shape)
raw_data.head()

(1000000, 6)


,id,content,title,media-type,source,published
0,f7ca322d-c3e8-40d2-841f-9d7250ac72ca,VETERANS saluted Worcester's first ever breakf...,Worcester breakfast club for veterans gives hu...,News,Redditch Advertiser,2015-09-07T10:16:14Z
1,609772bc-0672-4db5-8516-4c025cfd54ca,New Product Gives Marketers Access to Real Key...,Jumpshot Gives Marketers Renewed Visibility In...,News,Virtualization Conference & Expo,2015-09-17T15:00:00Z
2,1aa9d1b0-e6ba-4a48-ad0c-66552d896aac,Home »\rStyle » The Return Of The Nike Air Max...,The Return Of The Nike Air Max Sensation Has 8...,Blog,Streets Connect,2015-09-22T22:54:37Z
3,719699f9-47be-4bc7-969b-b53a881c95ae,NYMag.com Daily Intelligencer Vulture The Cut ...,This New Dating App Will Ruin Your Internet Game,Blog,The Cut,2015-09-16T23:12:11Z
4,a080f99a-07d9-47d1-8244-26a540017b7a,"KUALA LUMPUR, Sept 15 (MySinchew) -- The Kuala...",Pay up or face legal action: DBKL,News,My Sinchew,2015-09-15T10:17:53Z


 ## Text Preprocessing 

In [4]:
def Text_Cleaning(data, remove_numbers = True, custom_stopwords = None , remove_stop_words=True, lemmatize_words = False, stem_words= False):

    """

    Text Cleaning  
    Input: List of Documents    Output: Cleaned List of Documents
    Parameters: 
    custom_stopwords - Custom List of Stopwords(should be passed as a list)
    remove_stop_words - Binary parameter (True/False) to remove stopwords or do not want to perform this. Default is True
    lemmatize_words - Binary parameter (True/False) for Lemmatization (WordNet Lemmatizer used) or not. Default is False
    stem_words - Binary parameter (True/False) for Stemming (Lancaster Stemmer used) or not. Default is False

    """
    import os

    package = "nltk"

    try:
        __import__package
    except:
        os.system("pip install "+ package)
        import nltk
        nltk.download('stopwords')
        nltk.download('wordnet')
    
    import nltk
    import pandas as pd
    import string
    
    

    import re
    data = [re.sub("^\s+","",x) for x in data] # removes leading white spaces
    data = [re.sub("\s+\Z","",x) for x in data] # removes trailing white spaces

    data = [x.lower() for x in data]
    data = [x.translate(str.maketrans(' ',' ', '!"#$%&\'()*+,./:;<=>?@[\\]^`{|}~_')) for x in data]
    data = [x.replace('-', ' ') for x in data]
    
    if remove_numbers:
        data = [x.translate(str.maketrans('','', string.digits)) for x in data]


    ### Tokenizer
    def tokenize_word_text(text):
        """
        Sentence tokenize in NLTK with sent_tokenize 
        The sent_tokenize function uses an instance of NLTK known as PunktSentenceTokenizer
        This instance of NLTK has already been trained to perform tokenization on different European languages on the basis 
        of letters or punctuation that mark the beginning and end of sentences
        """
        from nltk.tokenize import WordPunctTokenizer 
        tokenizer = WordPunctTokenizer()
        return tokenizer.tokenize(text)


    ## Removing Stopwords
    if remove_stop_words:
        roman_numerals = ['iii', 'iiv', 'iiiv', 'vii', 'viii', 'xii', 'xiii', 'xxi','xxii', 'xxiii', 'xxv', 'xxvi', 'xxvii'] #exclude 2char numerals
        temporal_words = ['hour', 'day', 'week', 'month', 'quarter', 'year', 'annual', 'today', 'tomorrow', 'yesterday', 
                          'prior', 'prio', 'preceed', 'yet', "before", 'after', 'earlier', 'later', 'ago', 'this', 'next', 'last',
                          'once-daili', 'once-month', 'once-through', 'once-week', 'once', 'also',
                          'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday',
                          'jan', 'feb', 'apr', 'jun', 'jul', 'aug', 'sept', 'oct', 'nov', 'dec', 'may', 'onmay', 'postmay'
                         ]
        from nltk.corpus import stopwords
        stopword_list = stopwords.words('english')
        if custom_stopwords:
            stop_list = set(stopword_list + custom_stopwords + roman_numerals + temporal_words)
        else:
            stop_list = set(stopword_list + roman_numerals + temporal_words)

        def removeStopwords_before_tokens(text, stopwords):
            tokens = tokenize_word_text(text)
            return ' '.join([w for w in tokens if w not in stopwords])

        data = [removeStopwords_before_tokens(x, stop_list) for x in data]



    ##### Word Lemmatization Using WordNet
    def wordlemmatize(text):
        from nltk.stem.wordnet import WordNetLemmatizer 
        lem = WordNetLemmatizer()
        tokens = tokenize_word_text(text)
        return ' '.join([lem.lemmatize(w) for w in tokens])
    if lemmatize_words:
        data = [wordlemmatize(x) for x in data]



    ##### Word Stemming Using Lancaster Stemmer
    def wordstem(text):
        from nltk.stem.porter import PorterStemmer
        ls = PorterStemmer()
        tokens = tokenize_word_text(text)
        return ' '.join([ls.stem(w) for w in tokens])
    if stem_words:
        data = [wordstem(x) for x in data]


   
    return data

In [5]:
%%time
raw_data['clean_title'] = Text_Cleaning(raw_data['title'],lemmatize_words = True)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
CPU times: user 1min 58s, sys: 401 ms, total: 1min 59s
Wall time: 2min 6s


# News Filtering for topic Emission

Propsed Startegy : The objective of this part is to filter the news for a any topic. In this case we have only one topic 'Emission'. In order to get the topic releavnt news, I have built an independinet pipeline which can work for any topic.

** Working

-Create a topic relevant corpus by extracting relevant wikipedia pages.In order to understand what pages on wikipedia needs to be searched, use any pre-trained embedding model and get most similar words(i.e top 5) . The most similar words can be used for selection on wiki pages.

Once the corpus is built,use TF-IDF vectorizer to extract important words by filtering on TF-IDF scores . 
Now using any pre-trained embedding model calcualte the cosine similarity between the TF-IDF output words embeddings and Topic's most similar(top5 ) words. Words with coaines simialrity of more tha  60% are our shortlisted keyword list


## Topic relevant corpus

In [6]:
def Corpus_Generation(words,lemmatize=False,pos_tagging = ['NOUN','ADJ']):
    
    """

    
    Input: List of one or more words   Output: List of Document corresponding to each word
    
    Parameters: 
    lemmatize_words - Binary parameter (True/False) for Lemmatization (WordNet Lemmatizer used) or not. Default is False   
    pos_tagging - List of Part-of-speech tags(spaCy model used : 'en_core_web_sm') by using English POS tag set from https://spacy.io/api/annotation. Default is None
    
    

    """  
   
    if isinstance(words,str):
        theme_words  = []
        theme_words.append(words)
    else : theme_words = words  
        

    
    # Wikipedia page extraction
    theme_text = []
    for data1 in theme_words:
        try :
            theme_words = []        
            data = wikipedia.page(data1)
            print('Searching Wikipedia page for theme word : ',data1)
            print('Wikipedia page found : ',data.title)

            for token in nlp(data.content):
                t=token.text
                if lemmatize :           # Word Lemmatization
                    t = token.lemma_
                if pos_tagging:            # Part of Speech tagging
                    if token.pos_ in pos_tagging:                    
                        theme_words.append(t)
                else : theme_words.append(t)
            theme_text.append(" ".join(theme_words))       
            print('----------Corpus created---------------')
            print(str(" ".join(theme_words))[0:1000])
        except wikipedia.exceptions.PageError : print('!! ',data1,' not found on wikipedia.')
        except wikipedia.exceptions.DisambiguationError : print('!! ',data1,' not found on wikipedia.')
        except JSONDecodeError : print('!!  error on http server ')    
            
                                 
    return(theme_text)

In [7]:
nlp = spacy.load('en_core_web_sm')
import nltk
import string
import numpy as np    
from sklearn.feature_extraction.text import TfidfVectorizer        
 

## Extract wiki relevant pages using pre-trained google news model
pre-trained model location : https://code.google.com/archive/p/word2vec/

In [10]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/signalmedia/sample-1M.jsonl
/kaggle/input/signalmedia/GoogleNews-vectors-negative300.bin


In [ ]:
import gensim
google_pretrained_model = gensim.models.KeyedVectors.load_word2vec_format('/kaggle/input/signalmedia/GoogleNews-vectors-negative300.bin',binary=True)   
google_pretrained_model.most_similar('emission')

Output : 
    
[('emissions', 0.8727598190307617),
 ('carbon_emission', 0.7887347340583801),
 ('greenhouse_gas_emission', 0.7638275623321533),
 ('CO2_emission', 0.7512905597686768),
 ('emmissions', 0.7460930943489075),
 ('emission_reduction', 0.7448598146438599),
 ('carbon_dioxide_emission', 0.7386904954910278),
 ('Emission', 0.7283594608306885),
 ('greenhouse_gas_emissions', 0.7274933457374573),
 ('greenhouse_gas', 0.7139266729354858)]

In [ ]:
wiki_pages = ['Carbon footprint','Greenhouse gas','Emission_intensity','Vehicle_emissions_control','Emissions_trading','Carbon_emissions_reporting']

## Build corpus

In [ ]:
Theme_Corpus = Corpus_Generation(wiki_pages,lemmatize=True)

## TF-IDF filtering for imporatant words : 
This is done by calcualting tf-idf scores of our wiki generated corpus against 100 random wikipages. 

In [ ]:
if isinstance(Theme_Corpus,list):
    Topic_Corpus = " ".join(Theme_Corpus)

### tf_idf algorithm ------------------------------ starts :

#Function definition for TF-IDF Algorithm
def tf_idf(data,ngrams,lowercase,min_word_len):
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,ngrams),token_pattern=r'\b\w+\b',smooth_idf=False, lowercase =lowercase)
    tfidf=tfidf_vectorizer.fit_transform(data).toarray()
    final_tfidf=np.transpose(tfidf)
    final_tfidf= np.around( final_tfidf,decimals=3)
    final_tfidf=pd.DataFrame(final_tfidf,index=tfidf_vectorizer.get_feature_names())
    final_tfidf = pd.DataFrame(final_tfidf[len(final_tfidf.columns)-1]).reset_index()
    final_tfidf.columns = ['words', 'tfidf_score']
    final_tfidf['len'] = [len(x) for x in final_tfidf['words']]
    final_tfidf = final_tfidf.loc[final_tfidf.len>=min_word_len,['words','tfidf_score']]
    final_tfidf = final_tfidf.sort_values('tfidf_score',ascending=False)        
    return final_tfidf

print('---------------Performing TF-IDF calculations---------------')



# Perform TF-IDF with random 100 wikipedia pages 
random_page_names = [wikipedia.random(1) for i in range(100)]
random_page_names =set(random_page_names)
random_pages_content = []
for pages in random_page_names:
    #print(pages)
    theme_words = []
    try:
        data = wikipedia.page(pages)
        for token in nlp(data.content):
            t=token.text
            t = token.lemma_
            theme_words.append(t)
    except wikipedia.exceptions.DisambiguationError: theme_words = []
    except wikipedia.exceptions.PageError: theme_words = []  
    #except JSONDecodeError : 

    random_pages_content.append(" ".join(theme_words))      


In [ ]:
random_pages_content.append(Topic_Corpus)
final_tfidf_rnd = tf_idf(random_pages_content,ngrams=2,min_word_len=3,lowercase=True)

# Filtering words with tf-idf score >0 
filtered_unique_words = final_tfidf_rnd[final_tfidf_rnd.tfidf_score >0]

## Perfroming cosine similarity on word embeddings to get context similar words
Not all words are important to get relevant news, hence we will need only sematically similar words for filtering news

In [ ]:
# Loading pre-trained vocabulary and cleaning the words to find out if our tf-idf out words are available
model_vocab = pd.DataFrame(list(google_pretrained_model.vocab))
#cleaning words
model_vocab['clean_word'] = [x.translate(str.maketrans('','', '!"#$%&\'()*+,./:;<=>?@[\\]^`{|}~')).strip().lower() for x in model_vocab[0]]
model_vocab['model'] = 'google_news_ngrams_trained'

In [ ]:
# cleaning tf-df words in order to match against model vocabulary
tf_idf_words = list(set(filtered_unique_words.words))
k2 = pd.DataFrame(tf_idf_words)
k2['clean1'] = [str(seed).translate(str.maketrans('','', '!"#$%&\'()*+,./:;<=>?@[\\]^`{|}~')).strip().replace(' ','_').lower() for seed in tf_idf_words]
k2['clean2'] = [str(seed).translate(str.maketrans('','', '!"#$%&\'()*+,./:;<=>?@[\\]^`{|}~')).strip().replace(' ','').replace('_','').lower() for seed in tf_idf_words]
k2.columns = ['orig1','clean1','clean2']

In [ ]:
# Searching tf-idf word list in model vocab
w2v1 = model_vocab[model_vocab.clean_word.isin(k2.clean1)].reset_index(drop=True)
w2v1 = w2v1.drop_duplicates(['clean_word'], keep='first')
w2v11 = pd.merge(k2,w2v1,left_on='clean1',right_on='clean_word',how='left')

temp = w2v11.loc[w2v11.model.isna()==True,['orig1','clean1','clean2']].reset_index(drop=True)
w2v2  = model_vocab[model_vocab.clean_word.isin(temp.clean2)].reset_index(drop=True)
w2v2 = w2v2.drop_duplicates(['clean_word'], keep='first')
w2v22 = pd.merge(temp,w2v2,left_on='clean2',right_on='clean_word',how='left')

words_embed = w2v11[~(w2v11['model'].isna())].append(w2v22[~(w2v22['model'].isna())])
words_embed = words_embed.rename(columns={0:'model_orig_word'})

In [ ]:
# List of only those words whose embeddong is available
x = words_embed.drop_duplicates('model_orig_word')

In [ ]:
# Function for similarity matrix
def similarity_with_seed_words(words,seedwords,model):
    print('============Calculating Similarity scores===========')
    start_time = time.time()
    similarity_matrix=pd.DataFrame(index=words, columns=seedwords)
    for index,row in similarity_matrix.iterrows(): 
        #print(index)
        for column in similarity_matrix.columns:
            #print(column)
            try:
                similarity_matrix.loc[index,column]=model.similarity(column, index)  

            except KeyError:
                similarity_matrix.loc[index,column] = 0  
    print("--- %s seconds ---" , time.time() - start_time)                  
    return(similarity_matrix)  

In [ ]:
# Calculating similarity of embeddings
threshold_w2v = 0.5
words_w2v = []
seed_words = ['emissions','carbon_emission','greenhouse_gas_emission']

sim1 = similarity_with_seed_words(x.loc[x.model=='google_news_ngrams_trained','model_orig_word'],seed_words,google_pretrained_model)
for column in sim1.columns :
    words_w2v.extend(sim1[sim1[column] >= threshold_w2v].index)

keywords = set(words_w2v) 
print(len(keywords))

print(keywords) : 
{'CO2',
 'CO2_emission',
 'Carbon_Offset',
 'Clean_Air',
 'GHG',
 'GHG_emission',
 'NOx',
 'NOx_emission',
 'air_pollution',
 'carbon',
 'carbon_dioxide',
 'carbon_emission',
 'carbon_emissions',
 'carbon_footprint',
 'carbon_neutral',
 'carbon_offsetting',
 'climate_change',
 'emission',
 'emission_reduction',
 'emissions',
 'exhaust_emission',
 'fossil_fuel',
 'global_warming',
 'greenhouse_gas',
 'nitrogen_oxide',
 'particulate_emission',
 'particulate_matter',
 'pollutant',
 'polluter',
 'pollution',
 'renewable_energy',
 'smog',
 'soot',
 'sulfur_dioxide',
 'tailpipe_emission',
 'tailpipe_exhaust'}

## Filter News title with relevant keywords

In [ ]:
def incidence_matrix(docs,keywords,incidence_matrix=True):
    vectorizer1 = TfidfVectorizer(vocabulary=keywords,use_idf=False,ngram_range=(1,2),binary=incidence_matrix,lowercase=True,norm=False)
    tfidf = vectorizer1.fit_transform(docs).toarray()
    incidence_matrix = pd.DataFrame(tfidf,columns  = vectorizer1.get_feature_names())
    total_word_hit = incidence_matrix.sum(axis = 1)
    print('Word hit stats :')
    print(total_word_hit.value_counts())
    return(total_word_hit,incidence_matrix)

# Keyword hit
bd_hits,y1 = incidence_matrix(raw_data['clean_title'],keywords,incidence_matrix=True)
raw_data['keyword_hit'] = bd_hits
raw_data['keyword_hit'].value_counts()

In [ ]:
# Topic releavnt News filtering
filter_df = raw_data[raw_data.keyword_hit>=1].reset_index(drop=True)

# NER Tagging to extract the main company names 

In [ ]:
%%time

# NER tagging in Content
df1 = pd.DataFrame()

for i in range(filter_df.shape[0]-1):  
    print(i)
    for ent in nlp(filter_df.iloc[i,1]).ents:
        if len(ent.text) >0:
            x = pd.DataFrame([ent.text,ent.label_]).T
            x['id'] = filter_df.iloc[i,0]
            df1 = df1.append(x)
            

In [ ]:
df_content_orgs = df1[df1[1]=='ORG'].reset_index(drop=True)
df_content_orgs.columns = ['Company_Content','label','id']
df_content_orgs.shape

In [ ]:
# NER tagging in Title
df_title = pd.DataFrame()

for i in range(filter_df.shape[0]-1):  
    print(i)
    for ent in nlp(filter_df.iloc[i,2]).ents:
        if len(ent.text) >0:
            x = pd.DataFrame([ent.text,ent.label_]).T
            x['id'] = filter_df.iloc[i,0]
            df_title = df_title.append(x)


In [ ]:
df_title.columns = ['Company_Title','label','id']
df_title_orgs =  df_title[df_title['label']=='ORG'].reset_index(drop=True)
df_title_orgs.shape

## Assumption : Main company are only those tags which are simialr in content and title 

In [ ]:
companies_group = df_title_orgs.merge(df_content_orgs,on=['id'],how='inner')
companies_group['match']= companies_group['Company_Content'].str.lower()==companies_group['Company_Title'].str.lower()
companies_group = companies_group[companies_group['match']==True]
companies_group['Company_Name'] = companies_group['Company_Title'].str.lower()
unique_comps = companies_group.drop_duplicates(['id','Company_Name']).reset_index(drop=True)

In [ ]:
unique_companies_list = unique_comps['Company_Name'].unique()

# Company specific Text summarization using TextRank

In [ ]:
!pip install summa


In [ ]:
from summa import summarizer
news_combined = []
news_summary = []
for i in unique_companies_list :
    print(i)
    text = ' . '.join(filter_df.loc[filter_df['id'].isin(unique_comps.loc[unique_comps.Company_Name==i]['id'].unique()),'content'])
    news_combined.append(text)
    news_summary.append(summarizer.summarize(text))

In [ ]:
doc = nlp(news_summary[1])
displacy.serve(doc, style="ent")